# IMPORTS

In [ ]:
import pandas as pd
import numpy as np
import glob
import math
from math import radians, cos, sin, asin, sqrt
import os
from datetime import datetime,timedelta,timezone
import warnings
import xarray as xr
warnings.filterwarnings(action='ignore')


from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# FUNÇÕES

In [ ]:
def comb(a,b):
    c= []
    for i in a:
        for j in b:
            c.append([i,j])
    return c

def distance_to_point(ponto,par_coordenadas): # CÁLCULO DA DISTÂNCIA ENTRE PONTOS
    # Raio da terra em Km
    r = 6371
    # Cálculo das distâncias entre os pontos do Era5 e a torre
    lon1, lat1, lon2, lat2 = map(radians, [ ponto[1], ponto[0], par_coordenadas[1], par_coordenadas[0] ] )
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    hav = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    d = 2 * r * asin( sqrt(hav) )
    return float(d)

def distance_to_points(ponto,par_coordenadas):
    # Raio da terra em Km
    r = 6371
    # Vetor que irá armazenar as distâncias dos pontos do Era5 e da torre
    valor_distancia = []
    # Cálculo das distâncias entre os pontos do Era5 e a torre
    for par in par_coordenadas:
        lon1, lat1, lon2, lat2 = map(radians, [ ponto[1], ponto[0], par[1], par[0] ] )
        dlon = lon2 - lon1
        dlat = lat2 - lat1
        hav = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
        d = 2 * r * asin( sqrt(hav) )
        valor_distancia.append([float(par[0]),float(par[1]),float(d)])
    valor_distancia = pd.DataFrame(valor_distancia)
    valor_distancia.columns = ['latitude','longitude','distancias']

    return valor_distancia.astype(float)

def inter(x, y, x1, x2, y1, y2, Q11, Q21, Q12, Q22): # FUNÇÃO DE INTERPOLAÇÃO
  # x -> Latitude da torre
  # y -> Longitude da torre
  # x1 -> Latitude 1 da torre
  # x2 -> Latitude 2 da torre
  # y1 -> Longitude 1 da torre
  # y2 -> Longitude 2 da torre
  #
  # Q12          Q22
  #
  #    (Lat, Lon)
  #
  # Q11          Q21

  ans1 = ((((x2 - x) * (y2 - y)) / ((x2 - x1) * (y2 - y1)))) * Q11
  ans2 = ((((x - x1) * (y2 - y)) / ((x2 - x1) * (y2 - y1)))) * Q21
  ans3 = ((((x2 - x) * (y - y1)) / ((x2 - x1) * (y2 - y1)))) * Q12
  ans4 = ((((x - x1) * (y - y1)) / ((x2 - x1) * (y2 - y1)))) * Q22

  return ans1 + ans2 + ans3 + ans4

def pontos_proximos(ponto,resolucao):

    # ponto -> (Latitude, longitude)
    # Latitudes e Longitudes da Améria do Sul

    passo = resolucao
    latitude = np.arange(-57,13+passo,passo)
    longitude = np.arange(-82,-32+passo,passo)

    # Combinações de todas as latitudes e longitudes
    par_coordenadas = comb(latitude,longitude)

    # Cálculo das distâncias entre os pontos do Era5 e a torre
    valor_distancia  = distance_to_points(ponto,par_coordenadas)

    # Seleção dos 4 pontos Era5 mais próximos da torre
    pontos = valor_distancia.sort_values(by='distancias', ascending=True)[0:5]
    pontos = pontos.reset_index(drop=True)

    coordenadas = pd.DataFrame()
    # Testes necessários para selecionar os pontos bidimensionais

    coordenadas['latitude'] = pontos['latitude'].astype(float).unique()[:2]
    coordenadas['longitude'] = pontos['longitude'].astype(float).unique()[:2]

    lat = np.sort(list(np.around(coordenadas['latitude'],2)))
    lon = np.sort(list(np.around(coordenadas['longitude'],2)))

    lat1 = lat[0]
    lat2 = lat[1]
    lon1 = lon[0]
    lon2 = lon[1]

    return [lat1,lat2,lon1,lon2]

def normalizar_solar(row): # FUNÇÃO DE NORMALIZAR VARIÁVEIS SOLAR
    x= row/3600
    if (x > 1500):
        return 0
    elif (x<0):
        return 0
    else:
      return x

def salvar_dados(header,pasta,nome_arquivo,longo_prazo): # FUNÇÃO PARA SALVAMENTO DO TXT COM CABEÇALHO

  if not os.path.exists(pasta):
    os.makedirs(pasta)
  if os.path.exists('{}/{}.txt'.format(pasta,nome_arquivo)):
    os.remove('{}/{}.txt'.format(pasta,nome_arquivo))

  diferenca = timedelta(hours=-3)
  fuso_horario = timezone(diferenca)
  now = datetime.now().astimezone(fuso_horario)

  current_time = now.strftime("%d/%m/%y %H:%M:%S")
  header[0] ='Created {} by PrototipoMCP v1.2\n'.format(current_time)
  header_string=" ".join(map(str,header))

  longo_prazo.to_csv('{}/{}+_aux.txt'.format(pasta,nome_arquivo),sep='\t',encoding ='utf-8')

  archive = open('{}/{}.txt'.format(pasta,nome_arquivo), "a+",encoding ='utf-8')
  archive.write(header_string)

  archive_aux = open('{}/{}+_aux.txt'.format(pasta,nome_arquivo), "r",encoding ='utf-8')
  archive.write(archive_aux.read())

  archive.close()
  archive_aux.close()

  os.remove('{}/{}+_aux.txt'.format(pasta,nome_arquivo))

def extracao_era(latitude,longitude,nome_torre,nome_projeto,ano_inicial,ano_final):

  lat = float(math.floor(latitude / 10) * 10)

  lon = float(math.floor(longitude / 10) * 10)

  OUTPUT = pontos_proximos([latitude,longitude],0.25)

  data_all = pd.DataFrame()
  for ano in np.arange(ano_inicial,ano_final+1):
    print(f'Extraindo variáveis do ano: {ano}')
    for var in ['METEO','EOLICA', 'SOLAR']:
      ncFile = xr.open_dataset(f'/content/drive/Shareddrives/WRF/ERA1950/grib/{ano}/{ano}_era5_{var}{lat}{lon}.nc')
      data = ncFile.sel(latitude=OUTPUT[:2],longitude=OUTPUT[2:]).to_dataframe()
      data = pd.melt(data.reset_index(),id_vars=['time', 'longitude', 'latitude'], var_name='name', value_name='value')
      data_all = pd.concat([data_all,data])
      ncFile.close()

  data_all['longitude'] = data_all['longitude'].astype(float).round(2)
  data_all['latitude'] = data_all['latitude'].astype(float).round(2)
  data_all['time'] = pd.to_datetime(data_all['time'],format='%Y-%m-%d %H:%M:%S')

  # SEPARAÇÃO DOS DATAFRAMES DOS 4 PONTOS PRÓXMIOS
  Q11 = data_all.loc[(data_all['longitude']==OUTPUT[2]) & (data_all['latitude']==OUTPUT[0])].pivot(index='time', columns='name', values='value').astype(float)
  Q21 = data_all.loc[(data_all['longitude']==OUTPUT[2]) & (data_all['latitude']==OUTPUT[1])].pivot(index='time', columns='name', values='value').astype(float)
  Q12 = data_all.loc[(data_all['longitude']==OUTPUT[3]) & (data_all['latitude']==OUTPUT[0])].pivot(index='time', columns='name', values='value').astype(float)
  Q22 = data_all.loc[(data_all['longitude']==OUTPUT[3]) & (data_all['latitude']==OUTPUT[1])].pivot(index='time', columns='name', values='value').astype(float)

  # ARMAZENAMENTO DE INFORMAÇÕES SOBRE OS 4 PONTOS PRÓXIMOS - DISTANCIA E TESTE SE EXISTE ALGUM PONTO SEM DADOS
  df_datas = {}
  df_datas['Q11'],df_datas['Q21'],df_datas['Q12'],df_datas['Q22'] = Q11,Q21,Q12,Q22
  df_info = pd.DataFrame(columns=['Cod','Distancia','Empty','Lat','Lon'])
  df_info['Cod'] = ['Q11','Q21','Q12','Q22']
  df_info['Distancia'] = [distance_to_point([latitude,longitude],[OUTPUT[0],OUTPUT[2]]),
                          distance_to_point([latitude,longitude],[OUTPUT[1],OUTPUT[2]]),
                          distance_to_point([latitude,longitude],[OUTPUT[0],OUTPUT[3]]),
                          distance_to_point([latitude,longitude],[OUTPUT[1],OUTPUT[3]])]
  df_info['Empty'] = [Q11.dropna().empty,Q21.dropna().empty,Q12.dropna().empty,Q22.dropna().empty]
  df_info['Lat'] = [OUTPUT[0],OUTPUT[1],OUTPUT[0],OUTPUT[1]]
  df_info['Lon'] = [OUTPUT[2],OUTPUT[2],OUTPUT[3],OUTPUT[3]]

  if (df_info['Empty']==True).all():
    print('')
    print('Não existe dados próximos desse ponto informado')

  # TESTE SE EXISTE ALGUM PONTO SEM INFORMÇÃO (CASO HAJA, SERÁ RETORNADO O PONTO MAIS PRÓXIMO)
  if (df_info['Empty']==True).any():
    df_info = df_info.loc[df_info['Empty']==False]
    pontos_era = df_info['Cod']
  else:
    ERA5_interpolado = Q11.copy()
    for colunas in Q11.columns:
      ERA5_interpolado[str(colunas)] = inter(latitude,longitude,OUTPUT[0],OUTPUT[1],OUTPUT[2],OUTPUT[3],
                                          Q11[colunas],Q21[colunas],Q12[colunas],Q22[colunas])
    df_datas['interpolado'] = ERA5_interpolado
    df_info.loc['interpolado','Cod'] = 'interpolado'
    df_info.loc['interpolado','Lat'] = latitude
    df_info.loc['interpolado','Lon'] = longitude
    pontos_era = df_info['Cod']

  for select_point in pontos_era:

    era = df_datas[select_point].copy()

    # CONVERSÃO DOS DADOS BRUTOS
    d2m = era['d2m']
    t2m = era['t2m']
    u100 = era['u100']
    v100 = era['v100']
    u10 = era['u10']
    v10 = era['v10']

    era['Humidity [%]'] = (100*(np.exp((17.625*(d2m-273.15))/(243.04+(d2m-273.15)))/np.exp((17.625*(t2m-273.15))/(243.04+(t2m - 273.15)))))
    era['Temperature [°C]'] = t2m-273.5

    era['ssrd_avanco'] = era['ssrd'].shift(periods=1, freq="H")
    era['ssrd_novo'] = era['ssrd']-era['ssrd_avanco']
    era['GHI [W/m²]'] = era['ssrd_novo'].apply(normalizar_solar)
    era['GHI [W/m²]'] = era['GHI [W/m²]'].shift(periods=-3, freq="H")
    era['GHI [W/m²]'] = era['GHI [W/m²]'].fillna(0)

    era['fdir_avanco'] = era['fdir'].shift(periods=1, freq="H")
    era['fdir_novo'] = era['fdir']-era['fdir_avanco']
    era['DNI [W/m²]'] = era['fdir_novo'].apply(normalizar_solar)
    era['DNI [W/m²]'] = era['DNI [W/m²]'].shift(periods=-3, freq="H")
    era['DNI [W/m²]'] = era['DNI [W/m²]'].fillna(0)

    era["Speed_100m [m/s]"] = np.sqrt(u100 ** 2 + v100 ** 2)
    era["Direction_100m [°]"] = (57.3*np.arctan2(u100, v100)+180)
    era["Speed_10m [m/s]"] = np.sqrt(u10 ** 2 + v10 ** 2)
    era["Direction_10m [°]"] = (57.3*np.arctan2(u10, v10)+180)

    era['Pressure [hPa]'] = era['sp']/100
    era = era.drop(["u100","v100","u10","v10","t2m","sp","d2m","ssrd","ssrd_avanco","ssrd_novo","fdir","fdir_avanco","fdir_novo"],axis=1)
    era_10min = era.resample('10 min').mean().interpolate(method='linear').round(2)

    # SALVAMENTO DO PONTO
    header = [
    'Created 21/10/22 14:59:20 by MCP v1.2\n',
    '\n',
    'Latitude = {}\n'.format(df_info.loc[df_info['Cod']==select_point,'Lat'].values[0]),
    'Longitude = {}\n'.format(df_info.loc[df_info['Cod']==select_point,'Lon'].values[0]),
    'Elevation = \n',
    'Calm threshold = \n',
    '\n',
    'Included flags: <Unflagged data>, Synthesized\n',
    'Excluded flags: \n',
    '\n',
    'Time stamps indicate the beginning of the time step.\n',
    '\n',
    ]

    pasta = f'/content/drive/Shareddrives/TORRES_INFO/Resultados/{nome_projeto}/3.REGRESSAO/1.FONTES_LONGO_PRAZO/ERA5'
    nome_arquivo = f'ERA5_{select_point}_hour_{nome_torre}'
    salvar_dados(header,pasta,nome_arquivo,era.round(2))

    nome_arquivo = f'ERA5_{select_point}_10min_{nome_torre}'
    salvar_dados(header,pasta,nome_arquivo,era_10min)

    print('')
    print(f'O ponto {select_point} foi salvo no caminho: {pasta}/')

def extracao_eraland(latitude,longitude,nome_torre,nome_projeto,ano_inicial,ano_final):

  lat = float(math.floor(latitude / 10) * 10)

  lon = float(math.floor(longitude / 10) * 10)

  OUTPUT = pontos_proximos([latitude,longitude],0.1)

  data_all = pd.DataFrame()
  for ano in np.arange(ano_inicial,ano_final+1):
    print(f'Extraindo variáveis do ano: {ano}')
    for var in ['EOLICAV','EOLICAU', 'TEMP', 'SOLAR', 'PRESSURE', 'DEWPOINT']:
      ncFile = xr.open_dataset(f'/content/drive/Shareddrives/WRF/ERA_LAND_1950/grib/{ano}/{ano}_era5land_{var}{lat}{lon}.nc')
      data = ncFile.sel(latitude=OUTPUT[:2],longitude=OUTPUT[2:]).to_dataframe()
      # print(data)
      data['name'] = data.columns[0]
      data.columns = ['value','name']
      data_all = pd.concat([data_all,data])
      ncFile.close()

  data_all = data_all.reset_index()
  data_all['longitude'] = data_all['longitude'].astype(float).round(2)
  data_all['latitude'] = data_all['latitude'].astype(float).round(2)
  data_all['time'] = pd.to_datetime(data_all['time'],format='%Y-%m-%d %H:%M:%S')

  # SEPARAÇÃO DOS DATAFRAMES DOS 4 PONTOS PRÓXMIOS
  Q11 = data_all.loc[(data_all['longitude']==OUTPUT[2]) & (data_all['latitude']==OUTPUT[0])].pivot(index='time', columns='name', values='value').astype(float)
  Q21 = data_all.loc[(data_all['longitude']==OUTPUT[2]) & (data_all['latitude']==OUTPUT[1])].pivot(index='time', columns='name', values='value').astype(float)
  Q12 = data_all.loc[(data_all['longitude']==OUTPUT[3]) & (data_all['latitude']==OUTPUT[0])].pivot(index='time', columns='name', values='value').astype(float)
  Q22 = data_all.loc[(data_all['longitude']==OUTPUT[3]) & (data_all['latitude']==OUTPUT[1])].pivot(index='time', columns='name', values='value').astype(float)

  # ARMAZENAMENTO DE INFORMAÇÕES SOBRE OS 4 PONTOS PRÓXIMOS - DISTANCIA E TESTE SE EXISTE ALGUM PONTO SEM DADOS
  df_datas = {}
  df_datas['Q11'],df_datas['Q21'],df_datas['Q12'],df_datas['Q22'] = Q11,Q21,Q12,Q22
  df_info = pd.DataFrame(columns=['Cod','Distancia','Empty','Lat','Lon'])
  df_info['Cod'] = ['Q11','Q21','Q12','Q22']
  df_info['Distancia'] = [distance_to_point([latitude,longitude],[OUTPUT[0],OUTPUT[2]]),
                          distance_to_point([latitude,longitude],[OUTPUT[1],OUTPUT[2]]),
                          distance_to_point([latitude,longitude],[OUTPUT[0],OUTPUT[3]]),
                          distance_to_point([latitude,longitude],[OUTPUT[1],OUTPUT[3]])]
  df_info['Empty'] = [Q11.dropna().empty,Q21.dropna().empty,Q12.dropna().empty,Q22.dropna().empty]
  df_info['Lat'] = [OUTPUT[0],OUTPUT[1],OUTPUT[0],OUTPUT[1]]
  df_info['Lon'] = [OUTPUT[2],OUTPUT[2],OUTPUT[3],OUTPUT[3]]

  if (df_info['Empty']==True).all():
    print('')
    print('Não existe dados próximos desse ponto informado')
    exit

  # TESTE SE EXISTE ALGUM PONTO SEM INFORMÇÃO (CASO HAJA, SERÁ RETORNADO O PONTO MAIS PRÓXIMO)
  if (df_info['Empty']==True).any():
    print('')
    print('Foi puxado o ponto mais próximo')
    print('')
    df_info = df_info.loc[df_info['Empty']==False]
    cod = df_info.sort_values(by='Distancia')['Cod'].values[0]
    ponto_exportado = [df_info.sort_values(by='Distancia')['Lat'].values[0],df_info.sort_values(by='Distancia')['Lon'].values[0]]
    ERA5_interpolado = df_datas[cod]
  else:
    print('')
    print('Foi interpolado entre os 4 pontos mais próximos')
    print('')
    ERA5_interpolado = Q11.copy()
    for colunas in Q11.columns:
      ERA5_interpolado[str(colunas)] = inter(latitude,longitude,OUTPUT[0],OUTPUT[1],OUTPUT[2],OUTPUT[3],
                                          Q11[colunas],Q21[colunas],Q12[colunas],Q22[colunas])
    ponto_exportado = [latitude,longitude]

  # CONVERSÃO DOS DADOS BRUTOS
  d2m = ERA5_interpolado['d2m']
  t2m = ERA5_interpolado['t2m']
  u10 = ERA5_interpolado['u10']
  v10 = ERA5_interpolado['v10']

  ERA5_interpolado['Humidity [%]'] = (100*(np.exp((17.625*(d2m-273.15))/(243.04+(d2m-273.15)))/np.exp((17.625*(t2m-273.15))/(243.04+(t2m - 273.15)))))
  ERA5_interpolado['Temperature [°C]'] = t2m-273.5

  ERA5_interpolado['ssrd_avanco'] = ERA5_interpolado['ssrd'].shift(periods=1, freq="H")
  ERA5_interpolado['ssrd_novo'] = ERA5_interpolado['ssrd']-ERA5_interpolado['ssrd_avanco']
  ERA5_interpolado['GHI [W/m²]'] = ERA5_interpolado['ssrd_novo'].apply(normalizar_solar)
  ERA5_interpolado['GHI [W/m²]'] = ERA5_interpolado['GHI [W/m²]'].shift(periods=-3, freq="H")
  ERA5_interpolado['GHI [W/m²]'] = ERA5_interpolado['GHI [W/m²]'].fillna(0)

  ERA5_interpolado["Speed_10m [m/s]"] = np.sqrt(u10 ** 2 + v10 ** 2)
  ERA5_interpolado["Direction_10m [°]"] = (57.3*np.arctan2(u10, v10)+180)
  ERA5_interpolado['Pressure [hPa]'] = ERA5_interpolado['sp']/100
  ERA5_interpolado = ERA5_interpolado.drop(["u10","v10","t2m","sp","d2m","ssrd","ssrd_avanco","ssrd_novo"],axis=1)
  ERA5_interpolado_10min = ERA5_interpolado.resample('10 min').mean().interpolate(method='linear').round(2)

  # SALVAMENTO DO PONTO
  header = [
  'Created 21/10/22 14:59:20 by MCP v1.2\n',
  '\n',
  'Latitude = {}\n'.format(ponto_exportado[0]),
  'Longitude = {}\n'.format(ponto_exportado[1]),
  'Elevation = \n',
  'Calm threshold = \n',
  '\n',
  'Included flags: <Unflagged data>, Synthesized\n',
  'Excluded flags: \n',
  '\n',
  'Time stamps indicate the beginning of the time step.\n',
  '\n',
  ]

  pasta = f'/content/drive/Shareddrives/TORRES_INFO/Resultados/{nome_projeto}/3.REGRESSAO/1.FONTES_LONGO_PRAZO/ERA5LAND'
  nome_arquivo = f'ERA5LAND_hour_{nome_torre}'
  salvar_dados(header,pasta,nome_arquivo,ERA5_interpolado.round(2))

  nome_arquivo = f'ERA5LAND_10min_{nome_torre}'
  salvar_dados(header,pasta,nome_arquivo,ERA5_interpolado_10min)

  print('')
  print(f'O ponto foi salvo no caminho: {pasta}/')

  display(ERA5_interpolado)


# ERA5

In [ ]:
# 1. TMA VA7432 (Os quatro pontos e o interpolado)
torre_referencia = 'VA_FAZENDA_FAMOSA'
nome_projeto = 'SOLARIG_FAMOSA'
latitude = -4.86027778
longitude = -37.33777778
ano_inicial = 2000 #1980
ano_final = 2023
# Período: 1980 até o dado mais recente de 2023;
extracao_era(latitude, longitude, torre_referencia, nome_projeto, ano_inicial, ano_final)

Extraindo variáveis do ano: 2000
Extraindo variáveis do ano: 2001
Extraindo variáveis do ano: 2002
Extraindo variáveis do ano: 2003
Extraindo variáveis do ano: 2004
Extraindo variáveis do ano: 2005
Extraindo variáveis do ano: 2006
Extraindo variáveis do ano: 2007
Extraindo variáveis do ano: 2008
Extraindo variáveis do ano: 2009
Extraindo variáveis do ano: 2010
Extraindo variáveis do ano: 2011
Extraindo variáveis do ano: 2012
Extraindo variáveis do ano: 2013
Extraindo variáveis do ano: 2014
Extraindo variáveis do ano: 2015
Extraindo variáveis do ano: 2016
Extraindo variáveis do ano: 2017
Extraindo variáveis do ano: 2018
Extraindo variáveis do ano: 2019
Extraindo variáveis do ano: 2020
Extraindo variáveis do ano: 2021
Extraindo variáveis do ano: 2022
Extraindo variáveis do ano: 2023

O ponto Q11 foi salvo no caminho: /content/drive/Shareddrives/TORRES_INFO/Resultados/SOLARIG_FAMOSA/3.REGRESSAO/1.FONTES_LONGO_PRAZO/ERA5/

O ponto Q21 foi salvo no caminho: /content/drive/Shareddrives/TORRE

# ERA5LAND

In [ ]:
# 1. TMA_8538 (Os quatro pontos e o interpolado)
torre_referencia = 'VA_FAZENDA_FAMOSA'
nome_projeto = 'SOLARIG_FAMOSA'
latitude = -4.86027778
longitude = -37.33777778
ano_inicial = 2000 #1980
ano_final = 2023
# Período: 1980 até o dado mais recente de 2023;
extracao_eraland( latitude, longitude, torre_referencia, nome_projeto, ano_inicial, ano_final)

Extraindo variáveis do ano: 2000
Extraindo variáveis do ano: 2001
Extraindo variáveis do ano: 2002
Extraindo variáveis do ano: 2003
Extraindo variáveis do ano: 2004
Extraindo variáveis do ano: 2005
Extraindo variáveis do ano: 2006
Extraindo variáveis do ano: 2007
Extraindo variáveis do ano: 2008
Extraindo variáveis do ano: 2009
Extraindo variáveis do ano: 2010
Extraindo variáveis do ano: 2011
Extraindo variáveis do ano: 2012
Extraindo variáveis do ano: 2013
Extraindo variáveis do ano: 2014
Extraindo variáveis do ano: 2015
Extraindo variáveis do ano: 2016
Extraindo variáveis do ano: 2017
Extraindo variáveis do ano: 2018
Extraindo variáveis do ano: 2019
Extraindo variáveis do ano: 2020
Extraindo variáveis do ano: 2021
Extraindo variáveis do ano: 2022
Extraindo variáveis do ano: 2023

Foi puxado o ponto mais próximo


O ponto foi salvo no caminho: /content/drive/Shareddrives/TORRES_INFO/Resultados/ARACATI/3.REGRESSAO/1.FONTES_LONGO_PRAZO/ERA5LAND/


name,Humidity [%],Temperature [°C],GHI [W/m²],Speed_10m [m/s],Direction_10m [°],Pressure [hPa]
time,,,,,,
2000-01-01 00:00:00,79.875449,26.349701,0.0,7.234901,77.147300,1004.278672
2000-01-01 01:00:00,79.939820,26.180511,0.0,7.313914,83.922168,1004.354297
2000-01-01 02:00:00,80.444033,25.947388,0.0,7.309428,88.906456,1004.270313
2000-01-01 03:00:00,81.863719,25.567749,0.0,7.262745,92.428999,1003.917422
2000-01-01 04:00:00,83.972107,25.181427,0.0,6.950464,96.225087,1003.463828
...,...,...,...,...,...,...
2023-12-31 19:00:00,69.065082,29.072571,0.0,5.896515,35.398865,1005.471484
2023-12-31 20:00:00,73.581081,28.311584,0.0,5.891211,40.678685,1005.797656
2023-12-31 21:00:00,78.063801,27.535095,0.0,5.429019,43.906362,1006.092891
